In [1]:
import pandas as pd
import numpy as np
import os
import random
from copy import deepcopy
import math
from itertools import product
import pickle
import joblib

import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay

In [29]:
user_df = pd.read_csv('./data/user_spec.csv')
loan_df = pd.read_csv('./data/loan_result.csv')

In [30]:
merged_df=pd.merge(user_df, loan_df, on='application_id')

In [52]:
merged_df.isnull().sum()[merged_df.isnull().sum()!=0]

birth_year                               128096
gender                                   128096
credit_score                            1509276
yearly_income                                 6
company_enter_month                      400337
personal_rehabilitation_yn              5888701
personal_rehabilitation_complete_yn    11793977
existing_loan_cnt                       2685709
existing_loan_amt                       3890163
loan_limit                                 7382
loan_rate                                  7382
is_applied                              3257239
dtype: int64

In [34]:
df = merged_df[merged_df['personal_rehabilitation_complete_yn'].isnull()]['personal_rehabilitation_yn']
len(df)

11793977

In [36]:
df.isnull().sum(), (df==0).sum(), (df==1).sum(), df.isnull().sum()+ (df==0).sum()==len(df)

(5888701, 5905276, 0, True)

In [37]:
df = merged_df[merged_df['existing_loan_amt'].isnull()]['existing_loan_cnt']
len(df)

3890163

In [38]:
df.isnull().sum(), (df==0).sum(), (df>0).sum(), df.isnull().sum()+(df>0).sum() == len(df)

(2685709, 0, 1204454, True)

In [40]:
df = merged_df[merged_df['yearly_income'].isnull()]['income_type']
len(df)

6

In [50]:
df

5781948    OTHERINCOME
5781949    OTHERINCOME
5781950    OTHERINCOME
5781951    OTHERINCOME
5781952    OTHERINCOME
5781953    OTHERINCOME
Name: income_type, dtype: object

In [54]:
merged_df[merged_df['yearly_income'].isnull()]

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,...,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
5781948,341149,670502,1981.0,1.0,2022-06-30 21:32:18,930.0,NaN,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 21:32:20,64,254,45000000.0,10.3,NaN
5781949,341149,670502,1981.0,1.0,2022-06-30 21:32:18,930.0,NaN,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 21:32:19,13,262,35000000.0,16.4,NaN
5781950,341149,670502,1981.0,1.0,2022-06-30 21:32:18,930.0,NaN,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 21:32:19,38,113,5000000.0,19.5,NaN
5781951,341149,670502,1981.0,1.0,2022-06-30 21:32:18,930.0,NaN,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 21:32:19,14,128,15000000.0,12.5,NaN
5781952,341149,670502,1981.0,1.0,2022-06-30 21:32:18,930.0,NaN,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 21:32:19,29,265,10000000.0,9.1,NaN
5781953,341149,670502,1981.0,1.0,2022-06-30 21:32:18,930.0,NaN,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 21:32:19,57,71,23000000.0,15.4,NaN


In [84]:
merged_df[merged_df['user_id'] == 670502]

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,...,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
5478320,1746224,670502,1981.0,1.0,2022-06-30 22:41:11,930.0,0.0,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 22:41:13,15,204,16000000.0,14.6,NaN
5478321,1746224,670502,1981.0,1.0,2022-06-30 22:41:11,930.0,0.0,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 22:41:12,55,25,31000000.0,13.9,NaN
5478322,1746224,670502,1981.0,1.0,2022-06-30 22:41:11,930.0,0.0,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 22:41:12,24,70,3000000.0,13.0,NaN
5478323,1746224,670502,1981.0,1.0,2022-06-30 22:41:11,930.0,0.0,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 22:41:12,13,262,35000000.0,16.4,NaN
5478324,1746224,670502,1981.0,1.0,2022-06-30 22:41:11,930.0,0.0,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 22:41:13,64,254,45000000.0,10.3,NaN
5478325,1746224,670502,1981.0,1.0,2022-06-30 22:41:11,930.0,0.0,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 22:41:12,57,71,23000000.0,15.4,NaN
5478326,1746224,670502,1981.0,1.0,2022-06-30 22:41:11,930.0,0.0,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 22:41:25,20,111,25000000.0,9.8,NaN
5478327,1746224,670502,1981.0,1.0,2022-06-30 22:41:11,930.0,0.0,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 22:41:12,35,29,5000000.0,11.8,NaN
5478328,1746224,670502,1981.0,1.0,2022-06-30 22:41:11,930.0,0.0,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 22:41:26,52,187,3000000.0,6.0,NaN
5478329,1746224,670502,1981.0,1.0,2022-06-30 22:41:11,930.0,0.0,OTHERINCOME,NaN,기타,...,0.0,0.0,1.0,3000000.0,2022-06-30 22:41:13,3,205,10000000.0,9.9,NaN


In [ ]:
merged_df

In [53]:
merged_df[merged_df['income_type']=='OTHERINCOME']

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,...,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
50,1614045,860410,1976.0,1.0,2022-06-07 11:29:05,590.0,45000000.0,OTHERINCOME,NaN,기타,...,0.0,NaN,1.0,20000000.0,2022-06-07 11:29:07,42,258,3000000.0,16.7,NaN
292,696416,648540,1965.0,1.0,2022-06-07 11:38:46,750.0,30000000.0,OTHERINCOME,NaN,기타,...,0.0,NaN,1.0,NaN,2022-06-07 11:38:48,21,196,5000000.0,13.0,NaN
293,696416,648540,1965.0,1.0,2022-06-07 11:38:46,750.0,30000000.0,OTHERINCOME,NaN,기타,...,0.0,NaN,1.0,NaN,2022-06-07 11:38:47,29,265,34000000.0,9.0,NaN
294,696416,648540,1965.0,1.0,2022-06-07 11:38:46,750.0,30000000.0,OTHERINCOME,NaN,기타,...,0.0,NaN,1.0,NaN,2022-06-07 11:38:47,1,1,22000000.0,13.9,NaN
295,696416,648540,1965.0,1.0,2022-06-07 11:38:46,750.0,30000000.0,OTHERINCOME,NaN,기타,...,0.0,NaN,1.0,NaN,2022-06-07 11:38:47,35,29,5000000.0,17.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13527214,1327066,151422,1955.0,1.0,2022-03-22 01:19:24,980.0,20000000.0,OTHERINCOME,NaN,기타,...,NaN,NaN,1.0,NaN,2022-03-22 01:19:26,64,254,3000000.0,6.7,0.0
13527215,1327066,151422,1955.0,1.0,2022-03-22 01:19:24,980.0,20000000.0,OTHERINCOME,NaN,기타,...,NaN,NaN,1.0,NaN,2022-03-22 01:19:26,6,36,30000000.0,13.1,0.0
13527216,1327066,151422,1955.0,1.0,2022-03-22 01:19:24,980.0,20000000.0,OTHERINCOME,NaN,기타,...,NaN,NaN,1.0,NaN,2022-03-22 01:19:25,42,258,51000000.0,8.9,0.0
13527217,1327066,151422,1955.0,1.0,2022-03-22 01:19:24,980.0,20000000.0,OTHERINCOME,NaN,기타,...,NaN,NaN,1.0,NaN,2022-03-22 01:19:26,14,128,8000000.0,6.9,0.0


In [56]:
merged_df.columns

Index(['application_id', 'user_id', 'birth_year', 'gender', 'insert_time',
       'credit_score', 'yearly_income', 'income_type', 'company_enter_month',
       'employment_type', 'houseown_type', 'desired_amount', 'purpose',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn',
       'existing_loan_cnt', 'existing_loan_amt', 'loanapply_insert_time',
       'bank_id', 'product_id', 'loan_limit', 'loan_rate', 'is_applied'],
      dtype='object')

In [59]:
merged_df.shape

(13527250, 23)

In [58]:
merged_df.drop(['loanapply_insert_time', 'product_id', 'loan_limit', 'loan_rate', 'is_applied'], axis=1).drop_duplicates().shape

(11342557, 18)

In [63]:
len(loan_df['application_id'].unique()), loan_df.shape

(968866, (13527363, 7))

In [67]:
merged_df.drop(['loanapply_insert_time', 'product_id', 'loan_limit', 'loan_rate', 'is_applied'], axis=1).drop_duplicates()

TypeError: drop_duplicates() got an unexpected keyword argument 'axis'

In [68]:
13527250-968866

12558384

In [70]:
user_df.shape

(1394216, 17)

In [79]:
len(user_df['application_id'].unique()), len(user_df['user_id'].unique())

(1394216, 405213)

In [80]:
tmp = user_df.drop(['application_id','insert_time'],axis=1)

In [82]:
len(tmp.drop_duplicates())

1102436

In [75]:
for col in tmp.columns:
    print(tmp[col].duplicated().sum())

989003
1394134
1394213
128706
1394124
1393519
1394209
1387496
1394211
1394211
1393657
1394199
1394213
1394213
1394115
1393213


In [83]:
a=pd.merge(user_df, loan_df, on='application_id')
a.isnull().sum()

application_id                                0
user_id                                       0
birth_year                               128096
gender                                   128096
insert_time                                   0
credit_score                            1509276
yearly_income                                 6
income_type                                   0
company_enter_month                      400337
employment_type                               0
houseown_type                                 0
desired_amount                                0
purpose                                       0
personal_rehabilitation_yn              5888701
personal_rehabilitation_complete_yn    11793977
existing_loan_cnt                       2685709
existing_loan_amt                       3890163
loanapply_insert_time                         0
bank_id                                       0
product_id                                    0
loan_limit                              